In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


1. paste the link
2. from link extract information
3. from information create .md in the format like before

In [ ]:
# Cell 2: Imports and GraphQL setup
import os
import re
import requests
from datetime import datetime

# LeetCode GraphQL endpoint and query
graphql_url = "https://leetcode.com/graphql"
graphql_query = '''
query getQuestionDetail($titleSlug: String!) {
  question(titleSlug: $titleSlug) {
    questionId
    title
    difficulty
  }
}
'''

# Cell 3: Helper functions
def get_problem_info(url):
    """
    Given a LeetCode problem URL, extract slug and fetch question details.
    """
    m = re.search(r'/problems/([^/]+)/', url)
    if not m:
        raise ValueError(f"Invalid LeetCode problem URL: {url}")
    slug = m.group(1)
    resp = requests.post(
        graphql_url,
        json={ 'query': graphql_query, 'variables': {'titleSlug': slug} },
        headers={ 'Content-Type': 'application/json', 'X-Requested-With': 'XMLHttpRequest' }
    )
    resp.raise_for_status()
    data = resp.json().get('data', {}).get('question')
    if not data:
        raise RuntimeError(f"Could not fetch details for '{slug}'")
    return data['questionId'], data['title'], data['difficulty']


def sanitize_filename(s):
    """Remove forbidden characters and replace spaces with hyphens."""
    s = re.sub(r'[\\/:*?"<>|]', '', s)
    return s.strip().replace(' ', '-')


def next_index(directory):
    """Scan existing 'N-*.md' files to find the next index."""
    max_i = 0
    for fname in os.listdir(directory):
        m = re.match(r'^(\d+)-', fname)
        if m:
            max_i = max(max_i, int(m.group(1)))
    return max_i + 1


def create_markdown_file(url, solution_code=None, output_dir='.'):
    """Generate a numbered markdown file for the given problem URL."""
    idx = next_index(output_dir)
    qid, title, diff = get_problem_info(url)
    date_solved = datetime.now().strftime('%d-%m-%Y')

    safe_title = sanitize_filename(title)
    filename = f"{idx}-{safe_title}.md"
    filepath = os.path.join(output_dir, filename)

    md_lines = [
        f"# [{idx}. {title}]({url})",  
        f"**Difficulty:** {diff}",  
        f"**Date Solved:** {date_solved}\n",  
        "# Solution:",
    ]
    if solution_code:
        md_lines.extend(["```python", solution_code, "```"])
    else:
        md_lines.append("_Add your solution code here_")

    with open(filepath, 'w', encoding='utf-8') as f:
        f.write("\n".join(md_lines))

    print(f"Generated {filepath}")

# Cell 4: Usage example
# Replace the URL and optionally provide a code string or path to a .py file
problem_url = "https://leetcode.com/problems/find-users-with-valid-e-mails/description/?envType=study-plan-v2&envId=30-days-of-pandas&lang=pythondata"
# Option A: Direct code
solution_code = '''none'''
create_markdown_file(problem_url, solution_code)

# Option B: Read code from a local file
# with open('my_solution.py') as f:
#     code_str = f.read()
# create_markdown_file(problem_url, code_str)


Generated .\8-Find-Users-With-Valid-E-Mails.md
